In [1]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('../데이터자료/3. 증가율 데이터 정리끝.csv', encoding='cp949')
df_data = pd.read_csv('../데이터자료/4. 이자보상배율 데이터.csv', encoding='cp949')

# 데이터 정제 및 결측치 처리
df_data.columns = df_data.columns.str.replace("   ", "")
df_data["[제조]* (정상)영업손익(보고서기재)(IFRS연결)"].fillna(df_data["[제조]* (정상)영업손익(보고서기재)(IFRS)"], inplace=True)
df_data["[제조]이자비용(IFRS연결)"].fillna(df_data["[제조]이자비용(IFRS)"], inplace=True)

# 이자보상배율 계산
df_data["이자보상배율"] = np.round(df_data["[제조]* (정상)영업손익(보고서기재)(IFRS연결)"] / df_data["[제조]이자비용(IFRS연결)"], 2)

# 필요한 컬럼만 선택
df_data_mod = df_data[["회사명", "거래소코드", "회계년도", "이자보상배율"]]

# 날짜 형식 통일
df_data_mod["회계년도"] = pd.to_datetime(df_data_mod["회계년도"])
df["회계년도"] = pd.to_datetime(df["회계년도"])

# 데이터 병합
merged_df = pd.merge(df, df_data_mod, how='left', on=["회사명", "거래소코드", "회계년도"])

# 이상치 및 무한값 처리
def replace_outliers(row):
    if row['상장폐지일'] is np.NaN:
        if row['이자보상배율'] in [np.inf, -np.inf, np.NaN]:
            return df_data_mod[df_data_mod["이자보상배율"] > 0]["이자보상배율"].median()
    else:
        if row['이자보상배율'] in [np.inf, -np.inf, np.NaN]:
            return df_data_mod[df_data_mod["이자보상배율"] < 0]["이자보상배율"].median()
    return row['이자보상배율']

merged_df['이자보상배율'] = merged_df.apply(replace_outliers, axis=1)

# 최종 데이터 저장
merged_df.to_csv('../데이터자료/5. 이자보상배율 완료_리팩토링.csv', encoding='cp949', index=False)


C:\Users\성현태\AppData\Local\Temp\ipykernel_24056\2396729287.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_data_mod["회계년도"] = pd.to_datetime(df_data_mod["회계년도"])
